 CARPRICE_ASSIGNMENT

In [139]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.feature_selection import VarianceThreshold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

In [141]:
#Load the dataset
data=pd.read_csv('CarPrice_Assignment.csv')
#display the first few rows
print(data.head())

   car_ID  symboling                   CarName fueltype aspiration doornumber  \
0       1          3        alfa-romero giulia      gas        std        two   
1       2          3       alfa-romero stelvio      gas        std        two   
2       3          1  alfa-romero Quadrifoglio      gas        std        two   
3       4          2               audi 100 ls      gas        std       four   
4       5          2                audi 100ls      gas        std       four   

       carbody drivewheel enginelocation  wheelbase  ...  enginesize  \
0  convertible        rwd          front       88.6  ...         130   
1  convertible        rwd          front       88.6  ...         130   
2    hatchback        rwd          front       94.5  ...         152   
3        sedan        fwd          front       99.8  ...         109   
4        sedan        4wd          front       99.4  ...         136   

   fuelsystem  boreratio  stroke compressionratio horsepower  peakrpm citympg  \

In [143]:
#chake data type and null values
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   car_ID            205 non-null    int64  
 1   symboling         205 non-null    int64  
 2   CarName           205 non-null    object 
 3   fueltype          205 non-null    object 
 4   aspiration        205 non-null    object 
 5   doornumber        205 non-null    object 
 6   carbody           205 non-null    object 
 7   drivewheel        205 non-null    object 
 8   enginelocation    205 non-null    object 
 9   wheelbase         205 non-null    float64
 10  carlength         205 non-null    float64
 11  carwidth          205 non-null    float64
 12  carheight         205 non-null    float64
 13  curbweight        205 non-null    int64  
 14  enginetype        205 non-null    object 
 15  cylindernumber    205 non-null    object 
 16  enginesize        205 non-null    int64  
 1

In [147]:
#chack missing values
print(data.isnull().sum())

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64


In [149]:
#check sum of duplicat
data.duplicated().sum()
data.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [151]:
data.shape

(205, 26)

In [115]:
# Handling missing values
# Fill missing numeric values with the mean
numeric_columns = data.select_dtypes(include=['float64', 'int64']).columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].mean())


In [153]:
# Drop rows with missing categorical values
categorical_columns = data.select_dtypes(include=['object']).columns
data.dropna(subset=categorical_columns, inplace=True)

In [155]:
# Splitting features and target
X = data.drop(columns=["price"]) # Replace "price" with your actual target column name
y = data["price"] # Target variable


In [157]:
# Initialize OneHotEncoder
onehot = OneHotEncoder(sparse_output=False)



In [159]:
# Apply OneHotEncoder on a categorical column
price_encoded = onehot.fit_transform(data[['price']]) # Replace 'price' with your target column name
price_columns = onehot.get_feature_names_out(['price']) # Get feature names for the encoded data

# Convert the result to a DataFrame for better readability
price_encoded_df = pd.DataFrame(price_encoded, columns=price_columns, index=data.index)

# Combine the encoded column back to the main DataFrame
data = pd.concat([data, price_encoded_df], axis=1)
data.drop(columns=['price'], inplace=True) # Drop the original 'price' column if necessary

In [97]:
# Use pd.get_dummies to encode categorical variables
categorical_columns = ['CarName', 'fueltype', 'aspiration'] # Replace with your actual column names
data = pd.get_dummies(data, columns=categorical_columns, drop_first=True)

# This method automatically handles encoding for all specified categorical columns


KeyError: "None of [Index(['CarName', 'fueltype', 'aspiration'], dtype='object')] are in the [columns]"

In [125]:
# Identify numerical columns (excluding encoded columns)
numerical_columns = ['horsepower', 'peakrpm', 'citympg', 'highwaympg'] # Update based on your dataset
scaler = StandardScaler()

# Scale the numerical columns
data[numerical_columns] = scaler.fit_transform(data[numerical_columns])


In [127]:
# Split features and target
X = data.drop(columns=['categorical_columns']) # Replace 'target_column' with the actual target variable name
y = data['numerical_columns']

# Perform train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

KeyError: "['categorical_columns'] not found in axis"

In [131]:
# Drop original categorical columns and concatenate encoded columns
X_train = pd.concat([X_train.drop(columns=categorical_columns), X_train_encoded], axis=1)
X_test = pd.concat([X_test.drop(columns=categorical_columns), X_test_encoded], axis=1)

NameError: name 'X_train' is not defined

In [67]:
#Initialize models
models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree Regressor": DecisionTreeRegressor(random_state=42),
    "Random Forest Regressor": RandomForestRegressor(random_state=42),
    "Gradient Boosting Regressor": GradientBoostingRegressor(random_state=42),
    "Support Vector Regressor": SVR()
}

# Train models
for name, model in models.items():
    model.fit(X_train, y_train)
    print(f"{name} trained successfully.")


ValueError: could not convert string to float: 'four'

In [129]:



# Feature importance for Random Forest
rf = RandomForestRegressor(random_state=42)
rf.fit(X_train, y_train)
importance = rf.feature_importances_

# Display feature importance
for feature, imp in zip(X.columns, importance):
    print(f"{feature}: {imp:.4f}")

NameError: name 'X_train' is not defined

In [137]:
# Example: Tuning Random Forest
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=3, scoring='r2')


NameError: name 'GridSearchCV' is not defined